In [33]:
import numpy as np

# Create monkey patches
np.float = float
np.int = int
np.object = object
np.bool = bool

In [53]:
import librosa.display
import utility_functions as utils
import importlib
from scipy.spatial.distance import jensenshannon
import pandas as pd
import numpy as np
import madmom
# Import the necessary processors
from madmom.features.tempo import TempoEstimationProcessor
from madmom.features.beats import RNNBeatProcessor
from madmom.processors import IOProcessor
from madmom.features.beats import RNNBeatProcessor, DBNBeatTrackingProcessor
from madmom.processors import SequentialProcessor
import matplotlib.pylab as plt
from glob import glob
import librosa
from scipy.stats import ttest_ind
import librosa.display
import seaborn as sns
import pandas as pd
import utility_functions as utils
import importlib
from scipy.stats import norm
import numpy, scipy, matplotlib.pyplot as plt, IPython.display as ipd
from IPython.display import Audio

importlib.reload(utils)

data = '/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Audio/test.json'

In [35]:
data_audio = utils.get_all_artist_audio(data)

In [36]:
# Here I am just loading the saved progress because calculating everything would take like 2 days but above you can see the utility functions I used to calculate the features.
df = pd.read_excel('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Code/Audio/output_w_tempos_and_local_tempos.xlsx', engine='openpyxl')

In [43]:
df.rename(columns={'FilePath': 'Path', 'FileName': 'Song', 'Release_Year': 'Release Year'}, inplace=True)
df['Release Year'] = df['Release Year'].astype(int)

In [44]:
df

In [45]:
data_audio['Release Year'] = data_audio['Release Year'].astype(int)

In [46]:
data_audio

In [92]:
result = pd.merge(df, data_audio, on=['Path', 'Song'], how='right')

In [93]:
result = result[['Path', 'Song', 'Tempo1', 'Tempo2']]

In [96]:
def process_audio(audio_file):
    # Processing pipeline: from raw audio to beat activations to tempo estimation
    pipeline = SequentialProcessor([
        RNNBeatProcessor(),
        TempoEstimationProcessor(fps=100)
    ])
    tempo_histogram = pipeline(audio_file)
    sorted_tempo_histogram = tempo_histogram[tempo_histogram[:, 1].argsort()[::-1]]
    top_two_tempos = sorted_tempo_histogram[:2, :]
    return top_two_tempos

In [97]:
from tqdm import tqdm  # Import tqdm for the progress bar functionality

# Assuming 'result' is your DataFrame
# Filtering the DataFrame to find rows where both 'Tempo1' and 'Tempo2' are NaN
nan_tempo_rows = result[result['Tempo1'].isna()]

# Initialize a tqdm progress bar over the DataFrame iterrows()
for index, row in tqdm(nan_tempo_rows.iterrows(), total=nan_tempo_rows.shape[0], desc='Processing Audio'):
    top_two_tempos = process_audio(row['Path'])  # Call your audio processing function
    # Update the DataFrame with new tempo values, assuming top_two_tempos returns an array with tempos
    result.at[index, 'Tempo1'] = top_two_tempos[0][0]
    result.at[index, 'Tempo2'] = top_two_tempos[0][1]

In [99]:
result

In [156]:
data_audio = utils.get_all_artist_audio(data)

In [157]:
data_audio

In [158]:
result_final = pd.merge(result, data_audio, on=['Path', 'Song'], how='right')

In [159]:
result_final = result_final[['Artist', 'Album', 'Song', 'Coast', 'Release Year', 'Tempo1', 'Tempo2', 'Duration (s)', 'Sample Rate (Hz)', 'Path']]

In [160]:
result_final

In [161]:
lyricses = []
for index, item in result_final.iterrows():
    lyrics_path = item['Path'].replace('mp3', 'txt')
    try:
        with open(lyrics_path, 'r', encoding='utf-8') as f:
            content = f.read()
            lyricses.append(content)
    except FileNotFoundError:
        print(lyrics_path)

In [163]:
result_final['Lyrics'] = lyricses

In [164]:
result_final

In [165]:
result_final[result_final['Tempo1'].isna()]

In [166]:
a = '/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Audio/east_coast/Public Enemy/Yo Bum Rush the Show/MPE.mp3'

In [167]:
t1, t2 = process_audio(a)

In [169]:
result_final.loc[442, 'Tempo1'] = t1[0]

In [170]:
result_final.loc[442, 'Tempo2'] = t2[0]

In [171]:
result_final

In [172]:
result_final.to_excel("baseline_data.xlsx", sheet_name="Sheet1", index=False)

In [173]:
result_final['Sample Rate (Hz)'].unique()